# `Predicting students' dropout and academic success`


## Project Topic

Going to college isn't for everyone. There are many factors that can be involved in one's decision to enroll into college such as financial problems, motivation, or location. For the students that do attend college, there are also many different factors that can determine the academic success for students. This can include a students' academic path, demographics, and social-economic status. 

This project aims to `predict` the dropout and success rate of a student based on the academic path, demographic, and social-economic status. The dataset being used is utilized to construct classification models that can predict students' likelihood of dropping out or achieving academic success. The ***classification*** problem is approached as a three-category task, with one of the classes being significantly more prevalent than the others.

## Goal

* Determine which academic path has the highest/lowest dropout and success rate
* See whether there is a significant correlation between students' demographic and dropout/success rate

## Data Source

This project uses the [Predict students' dropout and academic success](https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success) dataset from the UCI Machine Learning Repository.

This dataset is derived from multiple databases of a higher education institution and focuses on undergraduate students in various disciplines. These disciplines include agronomy, design, education, nursing, journalism, management, social service, and technologies. The dataset contains information about students' academic paths, demographics, socio-economic factors, and their academic performance at the end of the first and second semesters. 

The dataset was developed as part of a project aimed at addressing the issue of academic dropout and failure in higher education. The project employs machine learning techniques to identify students who are at risk of encountering difficulties early on in their academic journey. By identifying these students in advance, appropriate support strategies can be implemented. The dataset includes information about students' academic paths, demographics, and socio-economic factors available at the time of their enrollment.

The problem is formulated as a three-category classification task, considering three outcomes: dropout, enrollment, and graduation. This classification is performed at the end of the standard duration of the respective courses.

`Citation of the paper` (M.V.Martins, D. Tolledo, J. Machado, L. M.T. Baptista, V.Realinho. (2021) "Early prediction of student’s performance in higher education: a case study" Trends and Applications in Information Systems and Technologies, vol.1, in Advances in Intelligent Systems and Computing series. Springer. DOI: 10.1007/978-3-030-72657-7_16)

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.stats
import math
import csv

In [2]:
# Load in the dataset
data = pd.read_csv('data.csv', header=None)

#Preview of what the data looks like
data.head()

0
0  Marital status;Application mode;Application or...
1  1;17;5;171;1;1;122.0;1;19;12;5;9;127.3;1;0;0;1...
2  1;15;1;9254;1;1;160.0;1;1;3;3;3;142.5;1;0;0;0;...
3  1;1;5;9070;1;1;122.0;1;37;37;9;9;124.8;1;0;0;0...
4  1;17;2;9773;1;1;122.0;1;38;37;5;3;119.6;1;0;0;...

## Data Information

The data gathered is a mix of different types such as *discrete, continuous, and categorical*, and there is no missing values thoughout the whole data set. These are the attribute names with their type from the data collected:

- Marital status - `discrete`
- Application mode - `discrete`
- Application order - `discrete`
- Course - `discrete`
- Daytime/evening attendance - `discrete`
- Previous qualification - `discrete`
- Previous qualification (grade) - `continuous`
- Nacionality - `discrete`
- Mother's qualification - `discrete`
- Father's qualification - `discrete`
- Mother's occupation - `discrete`
- Father's occupation - `discrete`
- Admission grade - `continuous`
- Displaced - `discrete`
- Educational special needs - `discrete`
- Debtor - `discrete`
- Tuition fees up to date - `discrete`
- Gender - `discrete`
- Scholarship holder - `discrete`
- Age at enrollment - `discrete`
- International - `discrete`
- Curricular units 1st sem (credited) - `discrete`
- Curricular units 1st sem (enrolled) - `discrete`
- Curricular units 1st sem (evaluations) - `discrete`
- Curricular units 1st sem (approved) - `discrete`
- Curricular units 1st sem (grade) - `discrete`
- Curricular units 1st sem (without evaluations) - `discrete`
- Curricular units 2nd sem (credited) - `discrete`
- Curricular units 2nd sem (enrolled) - `discrete`
- Curricular units 2nd sem (evaluations) - `discrete`
- Curricular units 2nd sem (approved) - `discrete`
- Curricular units 2nd sem (grade) - `discrete`
- Curricular units 2nd sem (without evaluations) - `discrete`
- Unemployment rate	 - `continuous`
- Inflation rate - `continuous`
- GDP - `continuous`
- Target - `categorical`

## Dictionary of Dataset

This shows what each of the numbers in the dataset correpsonds to

In [3]:
# Read the CSV file
data = pd.read_csv('data.csv', delimiter=';')

# Define dictionaries for column formatting
marital_status_dict = {
    1: 'Single',
    2: 'Married',
    3: 'Widower',
    4: 'Divorced',
    5: 'Facto union',
    6: 'Legally separated'
}

application_mode_dict = {
    1: '1st phase - general contingent',
    2: 'Ordinance No. 612/93',
    5: '1st phase - special contingent (Azores Island)',
    7: 'Holders of other higher courses',
    10: 'Ordinance No. 854-B/99',
    15: 'International student (bachelor)',
    16: '1st phase - special contingent (Madeira Island)',
    17: '2nd phase - general contingent',
    18: '3rd phase - general contingent',
    26: 'Ordinance No. 533-A/99, item b2) (Different Plan)',
    27: 'Ordinance No. 533-A/99, item b3 (Other Institution)',
    39: 'Over 23 years old',
    42: 'Transfer',
    43: 'Change of course',
    44: 'Technological specialization diploma holders',
    51: 'Change of institution/course',
    53: 'Short cycle diploma holders',
    57: 'Change of institution/course (International)',
}

application_order_dict = {}

course_dict = {
    33: 'Biofuel Production Technologies',
    171: 'Animation and Multimedia Design',
    8014: 'Social Service (evening attendance)',
    9003: 'Agronomy',
    9070: 'Communication Design',
    9085: 'Veterinary Nursing',
    9119: 'Informatics Engineering',
    9130: 'Equinculture',
    9147: 'Management',
    9238: 'Social Service',
    9254: 'Tourism',
    9500: 'Nursing',
    9556: 'Oral Hygiene',
    9670: 'Advertising and Marketing Management',
    9773: 'Journalism and Communication',
    9853: 'Basic Education',
    9991: 'Management (evening attendance)'
}

attendance_dict = {
    1: 'Daytime',
    0: 'Evening'
}

previous_qualification_dict = {
    1: 'Secondary education',
    2: "Higher education - bachelor's degree",
    3: 'Higher education - degree',
    4: "Higher education - master's",
    5: 'Higher education - doctorate',
    6: 'Frequency of higher education',
    9: '12th year of schooling - not completed',
    10: '11th year of schooling - not completed',
    12: 'Other - 11th year of schooling',
    14: '10th year of schooling',
    15: '10th year of schooling - not completed',
    19: 'Basic education 3rd cycle (9th/10th/11th year) or equiv.',
    38: 'Basic education 2nd cycle (6th/7th/8th year) or equiv.',
    39: 'Technological specialization course',
    40: 'Higher education - degree (1st cycle)',
    42: 'Professional higher technical course',
    43: 'Higher education - master (2nd cycle)'
}

previous_qualification_grade_dict = {}

nationality_dict = {
    1: 'Portuguese',
    2: 'German',
    6: 'Spanish',
    11: 'Italian',
    13: 'Dutch',
    14: 'English',
    17: 'Lithuanian',
    21: 'Angolan',
    22: 'Cape Verdean',
    24: 'Guinean',
    25: 'Mozambican',
    26: 'Santomean',
    32: 'Turkish',
    41: 'Brazilian',
    62: 'Romanian',
    100: 'Moldova (Republic of)',
    101: 'Mexican',
    103: 'Ukrainian',
    105: 'Russian',
    108: 'Cuban',
    109: 'Colombian'
}

mother_qualification_dict = {
    1: 'Secondary Education - 12th Year of Schooling or Eq.',
    2: 'Higher Education - Bachelor\'s Degree',
    3: 'Higher Education - Degree',
    4: 'Higher Education - Master\'s',
    5: 'Higher Education - Doctorate',
    6: 'Frequency of Higher Education',
    9: '12th Year of Schooling - Not Completed',
    10: '11th Year of Schooling - Not Completed',
    11: '7th Year (Old)',
    12: 'Other - 11th Year of Schooling',
    14: '10th Year of Schooling',
    18: 'General commerce course',
    19: 'Basic Education 3rd Cycle (9th/10th/11th Year) or Equiv.',
    22: 'Technical-professional course',
    26: '7th year of schooling',
    27: '2nd cycle of the general high school course',
    29: '9th Year of Schooling - Not Completed',
    30: '8th year of schooling',
    34: 'Unknown',
    35: 'Can\'t read or write',
    36: 'Can read without having a 4th year of schooling',
    37: 'Basic education 1st cycle (4th/5th year) or equiv.',
    38: 'Basic Education 2nd Cycle (6th/7th/8th Year) or Equiv.',
    39: 'Technological specialization course',
    40: 'Higher education - degree (1st cycle)',
    41: 'Specialized higher studies course',
    42: 'Professional higher technical course',
    43: 'Higher Education - Master (2nd cycle)',
    44: 'Higher Education - Doctorate (3rd cycle)'
}

father_qualification_dict = {
    1: 'Secondary Education - 12th Year of Schooling or Eq.',
    2: 'Higher Education - Bachelor\'s Degree',
    3: 'Higher Education - Degree',
    4: 'Higher Education - Master\'s',
    5: 'Higher Education - Doctorate',
    6: 'Frequency of Higher Education',
    9: '12th Year of Schooling - Not Completed',
    10: '11th Year of Schooling - Not Completed',
    11: '7th Year (Old)',
    12: 'Other - 11th Year of Schooling',
    14: '10th Year of Schooling',
    18: 'General commerce course',
    19: 'Basic Education 3rd Cycle (9th/10th/11th Year) or Equiv.',
    22: 'Technical-professional course',
    26: '7th year of schooling',
    27: '2nd cycle of the general high school course',
    29: '9th Year of Schooling - Not Completed',
    30: '8th year of schooling',
    34: 'Unknown',
    35: 'Can\'t read or write',
    36: 'Can read without having a 4th year of schooling',
    37: 'Basic education 1st cycle (4th/5th year) or equiv.',
    38: 'Basic Education 2nd Cycle (6th/7th/8th Year) or Equiv.',
    39: 'Technological specialization course',
    40: 'Higher education - degree (1st cycle)',
    41: 'Specialized higher studies course',
    42: 'Professional higher technical course',
    43: 'Higher Education - Master (2nd cycle)',
    44: 'Higher Education - Doctorate (3rd cycle)'
}

mother_occupation_dict = {
    0: 'Student',
    1: 'Representatives of the Legislative Power and Executive Bodies, Directors, Directors and Executive Managers',
    2: 'Specialists in Intellectual and Scientific Activities',
    3: 'Intermediate Level Technicians and Professions',
    4: 'Administrative staff',
    5: 'Personal Services, Security and Safety Workers and Sellers',
    6: 'Farmers and Skilled Workers in Agriculture, Fisheries and Forestry',
    7: 'Skilled Workers in Industry, Construction and Craftsmen',
    8: 'Installation and Machine Operators and Assembly Workers',
    9: 'Unskilled Workers',
    10: 'Armed Forces Professions',
    90: 'Other Situation',
    99: '(blank)',
    122: 'Health professionals',
    123: 'Teachers',
    125: 'Specialists in information and communication technologies (ICT)',
    131: 'Intermediate level science and engineering technicians and professions',
    132: 'Technicians and professionals, of intermediate level of health',
    134: 'Intermediate level technicians from legal, social, sports, cultural and similar services',
    141: 'Office workers, secretaries in general and data processing operators',
    143: 'Data, accounting, statistical, financial services and registry-related operators',
    144: 'Other administrative support staff',
    151: 'Personal service workers',
    152: 'Sellers',
    153: 'Personal care workers and the like',
    171: 'Skilled construction workers and the like, except electricians',
    173: 'Skilled workers in printing, precision instrument manufacturing, jewelers, artisans and the like',
    175: 'Workers in food processing, woodworking, clothing and other industries and crafts',
    191: 'Cleaning workers',
    192: 'Unskilled workers in agriculture, animal production, fisheries and forestry',
    193: 'Unskilled workers in extractive industry, construction, manufacturing and transport',
    194: 'Meal preparation assistants'
}

father_occupation_dict = {
    0: 'Student',
    1: 'Representatives of the Legislative Power and Executive Bodies, Directors, Directors and Executive Managers',
    2: 'Specialists in Intellectual and Scientific Activities',
    3: 'Intermediate Level Technicians and Professions',
    4: 'Administrative staff',
    5: 'Personal Services, Security and Safety Workers and Sellers',
    6: 'Farmers and Skilled Workers in Agriculture, Fisheries and Forestry',
    7: 'Skilled Workers in Industry, Construction and Craftsmen',
    8: 'Installation and Machine Operators and Assembly Workers',
    9: 'Unskilled Workers',
    10: 'Armed Forces Professions',
    90: 'Other Situation',
    99: '(blank)',
    122: 'Health professionals',
    123: 'Teachers',
    125: 'Specialists in information and communication technologies (ICT)',
    131: 'Intermediate level science and engineering technicians and professions',
    132: 'Technicians and professionals, of intermediate level of health',
    134: 'Intermediate level technicians from legal, social, sports, cultural and similar services',
    141: 'Office workers, secretaries in general and data processing operators',
    143: 'Data, accounting, statistical, financial services and registry-related operators',
    144: 'Other administrative support staff',
    151: 'Personal service workers',
    152: 'Sellers',
    153: 'Personal care workers and the like',
    171: 'Skilled construction workers and the like, except electricians',
    173: 'Skilled workers in printing, precision instrument manufacturing, jewelers, artisans and the like',
    175: 'Workers in food processing, woodworking, clothing and other industries and crafts',
    191: 'Cleaning workers',
    192: 'Unskilled workers in agriculture, animal production, fisheries and forestry',
    193: 'Unskilled workers in extractive industry, construction, manufacturing and transport',
    194: 'Meal preparation assistants'
}
# ...not finished

# Format columns
data['Marital status'] = data['Marital status'].map(marital_status_dict)
data['Application mode'] = data['Application mode'].map(application_mode_dict)
data['Course'] = data['Course'].map(course_dict)
data['Daytime/evening attendance'] = data['Daytime/evening attendance'].map(attendance_dict)
data['Previous qualification'] = data['Previous qualification'].map(previous_qualification_dict)
data['Nacionality'] = data['Nacionality'].map(nationality_dict)
data['Mother qualification'] = data['Mother qualification'].map(mother_qualification_dict)
data['Father qualification'] = data['Father qualification'].map(father_qualification_dict)
data['Mother occupation'] = data['Mother occupation'].map(mother_occupation_dict)
data['Father occupation'] = data['Father occupation'].map(father_occupation_dict)
# ... not finished

# Print formatted data
print(data[['Marital status', 'Application mode', 'Application order', 'Course', 'Daytime/evening attendance' ,'Previous qualification', 'Previous qualification (grade)',
             'Nacionality', 'Mother qualification', 'Father qualification', 'Mother occupation', 'Father occupation', 'Admission grade', 'Displaced', 'Educational special needs', 
             'Debtor', 'Tuition fees up to date', 'Gender', 'Scholarship holder', 'Age at enrollment', 'International', 'Curricular units 1st sem (credited)', 
             'Curricular units 1st sem (enrolled)', 'Curricular units 1st sem (evaluations)', 'Curricular units 1st sem (approved)', 'Curricular units 1st sem (grade)',
             'Curricular units 1st sem (without evaluations)', 'Curricular units 2nd sem (credited)', 'Curricular units 2nd sem (enrolled)', 'Curricular units 2nd sem (evaluations)',
             'Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (grade)', 'Curricular units 2nd sem (without evaluations)', 'Unemployment rate', 'Inflation rate', 'GDP', 'Target']])

     Marital status                  Application mode  Application order   
0            Single    2nd phase - general contingent                  5  \
1            Single  International student (bachelor)                  1   
2            Single    1st phase - general contingent                  5   
3            Single    2nd phase - general contingent                  2   
4           Married                 Over 23 years old                  1   
...             ...                               ...                ...   
4419         Single    1st phase - general contingent                  6   
4420         Single    1st phase - general contingent                  2   
4421         Single    1st phase - general contingent                  1   
4422         Single    1st phase - general contingent                  1   
4423         Single            Ordinance No. 854-B/99                  1   

                                   Course Daytime/evening attendance   
0         Anima

## Data Cleaning



In [6]:
# Rename columns for better readability
column_names = {
    'Daytime/evening attendance': 'Attendance',
    'Previous qualification (grade)': 'Previous grade'
    # ... not finished
}

data = data.rename(columns=column_names)

# Handle missing values
data = data.fillna('N/A')

data.describe()

Application order  Previous grade  Admission grade    Displaced   
count        4424.000000     4424.000000      4424.000000  4424.000000  \
mean            1.727848      132.613314       126.978119     0.548373   
std             1.313793       13.188332        14.482001     0.497711   
min             0.000000       95.000000        95.000000     0.000000   
25%             1.000000      125.000000       117.900000     0.000000   
50%             1.000000      133.100000       126.100000     1.000000   
75%             2.000000      140.000000       134.800000     1.000000   
max             9.000000      190.000000       190.000000     1.000000   

       Educational special needs       Debtor  Tuition fees up to date   
count                4424.000000  4424.000000              4424.000000  \
mean                    0.011528     0.113698                 0.880651   
std                     0.106760     0.317480                 0.324235   
min                     0.000000     0.000000                 0.000000   
25%                     0.000000     0.000000                 1.000000   
50%                     0.000000     0.000000                 1.000000   
75%                     0.000000     0.000000                 1.000000   
max                     1.000000     1.000000                 1.000000   

            Gender  Scholarship holder  Age at enrollment  ...   
count  4424.000000         4424.000000        4424.000000  ...  \
mean      0.351718            0.248418          23.265145  ...   
std       0.477560            0.432144           7.587816  ...   
min       0.000000            0.000000          17.000000  ...   
25%       0.000000            0.000000          19.000000  ...   
50%       0.000000            0.000000          20.000000  ...   
75%       1.000000            0.000000          25.000000  ...   
max       1.000000            1.000000          70.000000  ...   

       Curricular units 1st sem (without evaluations)   
count                                     4424.000000  \
mean                                         0.137658   
std                                          0.690880   
min                                          0.000000   
25%                                          0.000000   
50%                                          0.000000   
75%                                          0.000000   
max                                         12.000000   

       Curricular units 2nd sem (credited)   
count                          4424.000000  \
mean                              0.541817   
std                               1.918546   
min                               0.000000   
25%                               0.000000   
50%                               0.000000   
75%                               0.000000   
max                              19.000000   

       Curricular units 2nd sem (enrolled)   
count                          4424.000000  \
mean                              6.232143   
std                               2.195951   
min                               0.000000   
25%                               5.000000   
50%                               6.000000   
75%                               7.000000   
max                              23.000000   

       Curricular units 2nd sem (evaluations)   
count                             4424.000000  \
mean                                 8.063291   
std                                  3.947951   
min                                  0.000000   
25%                                  6.000000   
50%                                  8.000000   
75%                                 10.000000   
max                                 33.000000   

       Curricular units 2nd sem (approved)  Curricular units 2nd sem (grade)   
count                          4424.000000                       4424.000000  \
mean                              4.435805                         10.230206   
std                               3.014764          

## Data Analysis